In [1]:
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')

# insert at 1, 0 is the script path (or '' in REPL)
#sys.path.insert(1, '../src/data/')

sys.path.insert(1, '../src/visualization/')
import visualize as vi

from tensorflow.keras.layers import (Dense,
                                     Dropout,
                                     Flatten,
                                     Conv2D,
                                     MaxPooling2D,
                                     Activation)
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import (ImageDataGenerator,
                                                  img_to_array)

In [2]:
TRAIN_PATH = '../data/raw/kaggle/Face Mask Dataset/Train/'
VALIDATION_PATH = '../data/raw/kaggle/Face Mask Dataset/Validation/'
TEST_PATH = '../data/raw/kaggle/Face Mask Dataset/Test/'
IMG_SIZE = 50
BATCH_SIZE = 16

In [3]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=30,
#                                    width_shift_range=0.1,
#                                    height_shift_range=0.1,
#                                    shear_range=0.1,
#                                    zoom_range=0.1,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest')

train_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    batch_size=BATCH_SIZE,
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    color_mode="rgb",
                                                    class_mode='binary',
                                                    shuffle = True)

Found 10000 images belonging to 2 classes.


In [4]:
# validation_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=30,
#                                    width_shift_range=0.1,
#                                    height_shift_range=0.1,
#                                    shear_range=0.1,
#                                    zoom_range=0.1,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)


validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_PATH,
                                                         batch_size=BATCH_SIZE,
                                                         target_size=(IMG_SIZE, IMG_SIZE),
                                                         color_mode='grayscale',
                                                         class_mode='binary',
                                                         shuffle = True)

Found 800 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(directory=TEST_PATH,
                                                  batch_size=BATCH_SIZE,
                                                  target_size=(IMG_SIZE, IMG_SIZE),
                                                  color_mode='grayscale',
                                                  class_mode='binary')

Found 992 images belonging to 2 classes.


In [6]:
# Train:
# 1600 with_mask images
# 1600 without_mask images
# 3200 total images

In [7]:
# Test:
# 873 with_mask images
# 889 without_mask images
# 1762 total images

In [8]:
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 1)
# INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 1)

In [9]:
# model = Sequential()

# model.add(Conv2D(256, (3, 3), input_shape=INPUT_SHAPE,
#                  activation='relu',
#                  kernel_regularizer=l2(0.005)))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3),
#                 activation='relu',
#                 kernel_regularizer=l2(0.005)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())  # Converts 3D feature maps to 1D feature vecs
# model.add(Dense(64,
#                 activation='relu',
#                 kernel_regularizer=l2(0.005)))
                    
# model.add(Dense(1, activation='sigmoid'))


# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.summary()

In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3),
                 input_shape=INPUT_SHAPE,
                 kernel_regularizer=regularizers.l2(1e-5),
                 bias_regularizer=regularizers.l2(1e-5),
                 activity_regularizer=regularizers.l2(1e-5)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(32, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5),
                 bias_regularizer=regularizers.l2(1e-5),
                 activity_regularizer=regularizers.l2(1e-5)))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, kernel_regularizer=regularizers.l2(1e-5),
                bias_regularizer=regularizers.l2(1e-5),
                activity_regularizer=regularizers.l2(1e-5)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7744)              0

In [11]:
#results = model.fit(train_generator,
#                    steps_per_epoch=2000 // BATCH_SIZE,
#                    epochs=2,
#                    validation_data=validation_generator,
#                    validation_steps=800 // BATCH_SIZE)
#model.save('hold-on-to-your-butts.h5')

# Only augmented data is passed into the model through the generator, never the original photo

In [12]:
results = model.fit(train_generator,
                    batch_size=BATCH_SIZE,
                    steps_per_epoch = 10000 // BATCH_SIZE,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps = 800 // BATCH_SIZE)
#model.save('hold-on-to-your-butts.h5')

# Only augmented data is passed into the model through the generator, never the original photo

Epoch 1/10


UnimplementedError:  Fused conv implementation does not support grouped convolutions for now.
	 [[node sequential/conv2d/BiasAdd (defined at <ipython-input-12-48838bcaf066>:6) ]] [Op:__inference_train_function_1355]

Function call stack:
train_function


In [ ]:
# aug = ImageDataGenerator(**data_gen_args)

# # train the network
# H = model.fit_generator(aug.flow(img_train, target_train, batch_size=image_train.shape[0]),
#     validation_data=(img_val, target_val), steps_per_epoch=image_train.shape[0] // BS,
#     epochs=EPOCHS)

In [ ]:
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
#file_path = "../models/50x50-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#
#early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
#callbacks_list = [checkpoint]

In [ ]:
model.evaluate(test_generator)

In [ ]:
# 10 epochs: [0.41691482067108154, 0.8471518754959106]

In [ ]:
# predictions = model.predict(test_generator)

In [ ]:
train_preds = model.predict(train_generator, batch_size=BATCH_SIZE, verbose=0)
flat_list = [item for sublist in train_preds.tolist() for item in sublist]
rounded_train_preds = [int(np.round(ele)) for ele in flat_list]

In [ ]:
cm = confusion_matrix(y_true=train_generator.classes, y_pred=rounded_train_preds)
cm

In [ ]:
valid_preds = model.predict(validation_generator, batch_size=BATCH_SIZE, verbose=0)
flat_list = [item for sublist in valid_preds.tolist() for item in sublist]
rounded_valid_preds = [int(np.round(ele)) for ele in flat_list]

In [ ]:
cm = confusion_matrix(y_true=validation_generator.classes, y_pred=rounded_valid_preds)
cm

In [ ]:
test_preds = model.predict(test_generator, batch_size=BATCH_SIZE, verbose=0)
flat_list = [item for sublist in test_preds.tolist() for item in sublist]
rounded_test_preds = [int(np.round(ele)) for ele in flat_list]

In [ ]:
cm = confusion_matrix(y_true=test_generator.classes, y_pred=rounded_test_preds)
cm

In [ ]:
vi.visualize_training_results(results)

In [ ]:
CATEGORIES = ['with_mask', 'without_mask']
vi.plot_confusion_matrix(cm=cm, classes=CATEGORIES, title='Confusion Matrix', normalize=False)

In [ ]:
# # Import image to test

# import cv2
# from tensorflow.keras.models import load_model

# def prepare(filepath, img_size=IMG_SIZE):
#     img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
#     new_array = cv2.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    
# model = load_model("the-model.model")

# prediction = model.predict([prepare('image.jpg')])